In [1]:
import pandas as pd
import re
import random
import json
from tqdm.auto import tqdm, trange

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/MyDrive/diploma/data

aligned_chekhov_75.000_sents_09_02.json		     aligned_konstitution_sents_09_02.json
aligned_chekhov_chameleon_sents_09_02.json	     aligned_land_direct_sents_09_02.json
aligned_chekhov_maria_ivanovna_sents_09_02.json      aligned_news_names_09_02.json
aligned_chekhov_person_in_case_sents_09_02.json      aligned_news_sents_09_02.json
aligned_chekhov_prishibaev_sents_09_02.json	     aligned_shchedrin_konyaga_sents_09_02.json
aligned_chekhov_self-delusion_sents_09_02.json	     aligned_shchedrin_vilage_fire_sents_09_02.json
aligned_chekhov_trifon_sents_09_02.json		     aligned_wikipedia_sents_09_02.json
aligned_furmanov_chapaev_sents_09_02.json	     all_dicts_data.tsv
aligned_gaydar_distant_countries_sents_09_02.json    all_phrases.tsv
aligned_gogol_dead_souls_sample_sents_09_02.json     e-mordovia
aligned_gorkiy_happiness_sents_09_02.json	     mdf_mono
aligned_gorkiy_hero_sents_09_02.json		     moksha_bible.tsv
aligned_gorkiy_italian_11_sents_09_02.json	     moksha_pravda.tsv
aligned_gork

In [4]:
PARSED_DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

In [5]:
ALIGNED_DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

## Load sentence-parallel data

### bible

In [6]:
bible_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'moksha_bible.tsv', sep='\t')
print(bible_df.shape)

bible_df = bible_df.dropna()
print(bible_df.shape)

bible_df = bible_df[~bible_df['mdf'].str.startswith('Глава ')]
print(bible_df.shape)
print(bible_df.columns)

(11650, 4)
(11633, 4)
(11583, 4)
Index(['Unnamed: 0', 'mdf', 'ru', 'source'], dtype='object')


In [7]:
bible_pairs = list(zip(bible_df['mdf'], bible_df['ru']))

print(len(bible_pairs))
print(random.choice(bible_pairs))

11583
('Тостонь паксяса сон няйсь эши и вакссонза учань и сявань колма стадат, конат ваймосесть тоса, сяс мес тя эшиста симнезь стадатнень. Эшись пякстафоль оцю кевса.', 'И увидел: вот, на поле колодезь, и там три стада мелкого скота, лежавшие около него, потому что из того колодезя поили стада. Над устьем колодезя был большой камень.')


### words and phrases

In [8]:
word_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'all_dicts_data.tsv', sep='\t')

assert not word_df.isna().sum().sum()

word_pairs = list(zip(word_df['mdf'], word_df['ru']))

print(len(word_pairs))
print(random.choice(word_pairs))

3589
('содамс', 'отгадать')


In [9]:
phrases_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'all_phrases.tsv', sep='\t')

assert not phrases_df.isna().sum().sum()

phrases_pairs = list(zip(phrases_df['mdf'], phrases_df['ru']))

print(len(phrases_pairs))
print(random.choice(phrases_pairs))

726
('В вашем селе есть школа?', 'Велесонт ули школа?')


### news

In [22]:
with open(ALIGNED_DATA_PATH_PREFIX + 'aligned_news_names_09_02.json', 'r') as f:
    news_names_df = pd.DataFrame(json.load(f))

assert not news_names_df.isna().sum().sum(), "Missing values detected in news_names"

news_names_pairs = list(zip(news_names_df['mdf'], news_names_df['ru']))

print(len(news_names_pairs))
print(random.choice(news_names_pairs))

4560
('Мордовия Республикань Конституциять шинц мархта Мордовиянь Правительствань Председательть поздравленияц', 'Поздравление Председателя Правительства Мордовии с Днём Конституции Республики Мордовия')


In [23]:
with open(ALIGNED_DATA_PATH_PREFIX + 'aligned_news_sents_09_02.json', 'r') as f:
    news_sents_df = pd.DataFrame(json.load(f))

assert not news_sents_df.isna().sum().sum(), "Missing values detected in news_sents"

news_sents_pairs = list(zip(news_sents_df['mdf'], news_sents_df['ru']))

print(len(news_sents_pairs))
print(random.choice(news_sents_pairs))

65970
('Артём Здунов китькстазе, што скверсь петевсь ошеряйхнень мяльснон кирдезь, ванфтомок природнай ланшафтть.', 'Артём Здунов подчеркнул, что сквер реконструировали с учетом пожеланий горожан, сохранив природный ландшафт.')


### wikisource

In [24]:
document_names = [
    'chekhov_75.000_sents',
    'chekhov_chameleon_sents',
    'chekhov_maria_ivanovna_sents',
    'chekhov_person_in_case_sents',
    'chekhov_prishibaev_sents',
    'chekhov_self-delusion_sents',
    'chekhov_trifon_sents',
    'furmanov_chapaev_sents',
    'gaydar_distant_countries_sents',
    'gogol_dead_souls_sample_sents',
    'gorkiy_happiness_sents',
    'gorkiy_hero_sents',
    'gorkiy_italian_11_sents',
    'gorkiy_mother_sents',
    'gorkiy_mother_part2_sents',
    'gorkiy_russian_fairy_tales_sents',
    'konstitution_sents',
    'land_direct_sents',
    'shchedrin_konyaga_sents',
    'shchedrin_vilage_fire_sents',
]

In [25]:
wikisource_pairs = {}

for doc_name in document_names:
    file_path = ALIGNED_DATA_PATH_PREFIX + f"aligned_{doc_name}_09_02.json"
    with open(file_path, 'r') as f:
        df = pd.DataFrame(json.load(f))

    assert not df.isna().sum().sum(), f"Missing values detected in {doc_name}"

    pairs = list(zip(df['mdf'], df['ru']))

    if doc_name == 'konstitution_sents':
        pairs = [pair for pair in pairs if not pair[1].startswith('Статья')]

    wikisource_pairs[doc_name] = pairs

    print(f"{doc_name}: {len(pairs)} pairs")
    print(random.choice(pairs))

    print()

chekhov_75.000_sents: 130 pairs
('Тинь, Неклюжефнень, кельктядязь, тейнть жертват тиендихть, а тяса сембе эряфцень работат, вачкият, кода калсь эйти...', 'Вас, Неклюжевых, любят, вам жертвуют, а тут всю жизнь работаешь, бьешься как рыба об лед... честен, как сама честность, и — хоть бы одна счастливая минута!')

chekhov_chameleon_sents: 92 pairs
('Азсак, што мон муйня и кучиня...', 'Скажешь, что я нашёл и прислал...')

chekhov_maria_ivanovna_sents: 27 pairs
('Ламос ащесь од ломанць пяк мазы авать инголе.', 'Долго стоял молодой человек перед прекрасной женщиной.')

chekhov_person_in_case_sents: 186 pairs
('— Беликовсь эрясь сяка жа кудса, коса и мон. — ушедсь тага Буркинць, — сяка жа этажса, кенкш каршеса кенкш, минь сидеста няендемя, и мон содалиня сонь куд-ёткса эряфонц.', '— Беликов жил в том же доме, где и я, — продолжал Буркин, — в том же этаже, дверь против двери, мы часто виделись, и я знал его домашнюю жизнь.')

chekhov_prishibaev_sents: 84 pairs
('Тон полицейскай урядникат, ды 

### wikipedia

In [26]:
with open(ALIGNED_DATA_PATH_PREFIX + 'aligned_wikipedia_sents_09_02.json', 'r') as f:
    wiki_sents_df = pd.DataFrame(json.load(f))

assert not wiki_sents_df.isna().sum().sum(), "Missing values detected in news_sents"

wiki_sents_pairs = list(zip(wiki_sents_df['mdf'], wiki_sents_df['ru']))

print(len(wiki_sents_pairs))
print(random.choice(wiki_sents_pairs))

1400
('Сяс видьмотне эрявихть видемс эстокиге, сяда пара шувару, ведень шокши модас.', 'Едят его обычно свежим; семена, по вкусу напоминающие каштаны, поджаривают.')


### Union

In [27]:
all_pairs = sorted({
    c for c in
    word_pairs + phrases_pairs + bible_pairs + news_names_pairs + news_sents_pairs + [pair for pairs in wikisource_pairs.values() for pair in pairs] + wiki_sents_pairs
    if c[0] and c[1]
})
print(len(all_pairs))

88748


In [28]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/all_pairs.json', 'w') as f:
    json.dump(all_pairs, f, ensure_ascii=False, indent=4)

# Select traindev data

In [29]:
print(len(set(bible_pairs)))

random.seed(1)
bible_devtest = random.sample(sorted(set(bible_pairs)), k=800)

11583


In [30]:
print(len(set(news_sents_pairs)))

random.seed(1)
news_sents_devtest = random.sample(sorted(set(news_sents_pairs)), k=1000)

62308


In [31]:
print(len(set(wikisource_pairs['furmanov_chapaev_sents'])))

random.seed(1)
furmanov_chapaev_devtest = random.sample(sorted(set(wikisource_pairs['furmanov_chapaev_sents'])), k=400)

756


In [32]:
print(len(set(wikisource_pairs['gorkiy_mother_sents'])))

random.seed(1)
gorkiy_mother_devtest = random.sample(sorted(set(wikisource_pairs['gorkiy_mother_sents'])), k=200)

1477


In [33]:
print(len(set(wikisource_pairs['gaydar_distant_countries_sents'])))

random.seed(1)
gaydar_distant_countries_devtest = random.sample(sorted(set(wikisource_pairs['gaydar_distant_countries_sents'])), k=200)

1100


In [34]:
components = {
    'bible': bible_devtest,
    'news_sents': news_sents_devtest,
    'gorkiy_mother': gorkiy_mother_devtest,
    'furmanov_chapaev': furmanov_chapaev_devtest,
    'gaydar_distant_countries': gaydar_distant_countries_devtest
}
dev = {k: v[:len(v)//2] for k, v in components.items()}
test = {k: v[len(v)//2:] for k, v in components.items()}

In [35]:
print(len({s for c in dev.values() for s in c}))
print(len({s for c in test.values() for s in c}))

1300
1300


In [36]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/dev.json', 'w') as f:
    json.dump(dev, f, ensure_ascii=False, indent=4)

In [37]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/test.json', 'w') as f:
    json.dump(test, f, ensure_ascii=False, indent=4)

In [38]:
devtest = {s for c in components.values() for s in c}
len(devtest)

2600

In [39]:
train = sorted(set(all_pairs).difference(devtest))
print(len(train))

86148


In [40]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/train.json', 'w') as f:
    json.dump(train, f, ensure_ascii=False, indent=4)